In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install kashgari==2.0.0

In [ ]:
import numpy as np
import kashgari
from kashgari.embeddings import BertEmbedding
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.tasks.labeling import BiGRU_CRF_Model
from kashgari.tasks.labeling import CNN_LSTM_Model
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.tasks.labeling import BiGRU_Model

file_path='train_2.txt'

In [ ]:
os.chdir("/content/drive/My Drive/AiCup-1") #更改路徑
os.getcwd() #查看當前路徑

'/content/drive/My Drive/AiCup-1'

In [ ]:
def loadInputFile(path):
    trainingset = list()  # store trainingset [content,content,...]
    position = list()  # store position [article_id, start_pos, end_pos, entity_text, entity_type, ...]
    mentions = dict()  # store mentions[mention] = Type
    with open(file_path, 'r', encoding='utf8') as f:
        file_text=f.read().encode('utf-8').decode('utf-8-sig')
    datas=file_text.split('\n\n--------------------\n\n')[:-1]
    for data in datas:
        data=data.split('\n')
        content=data[0]
        trainingset.append(content)
        annotations=data[1:]
        for annot in annotations[1:]:
            annot=annot.split('\t') #annot= article_id, start_pos, end_pos, entity_text, entity_type
            position.extend(annot)
            mentions[annot[3]]=annot[4]
    
    return trainingset, position, mentions

def list_to_matrix(Lst):
    data_num = int(len(Lst)/5)
    matrix = np.zeros((data_num , 5))
    matrix = matrix.astype('unicode')
    i=0
    for j in range (0,data_num):
        matrix[j,:] = Lst[i:i+5]
        i+=5
    
    return matrix


trainingset, position, mentions = loadInputFile(file_path)
data_matrix = list_to_matrix(position)

num_of_data_num = np.zeros((1,int(data_matrix[len(data_matrix)-1,0])))
for k in range (0,len(data_matrix)-1):
    num_of_data_num[0,int(data_matrix[k,0])-1] +=1

In [ ]:
for i in range (len(trainingset)):
    trainingset[i]=list(trainingset[i])
    train_y1=[]
    yp=[]
    yo=[]
    j=0
for j in range (len(trainingset)):
    for k in range (len(trainingset[j])):
        yo=k+1
    yp.append(yo)
    
for l in range (len(trainingset)):
    Q='O'*yp[l]
    Q=list(Q)
    train_y1.append(Q)    

In [ ]:
for r in range (0,int(len(position)),5):
    flag=0
    for h in range (int(position[r+1]),int(position[r+2]),1):
        if (flag==0):
            del train_y1[int(position[r])][h]
            train_y1[int(position[r])].insert(h,'B-' + position[r+4])
            flag=1
        else:
            del train_y1[int(position[r])][h]
            train_y1[int(position[r])].insert(h,'I-' + position[r+4])

In [ ]:
index = 0
sentence = 0
train_scli = []
y1_scli =[]

for m in range (len(trainingset)):
    flag = 0
    for n in range (len(trainingset[m])):
        if (n - index) >= 100:
                train_scli.append(trainingset[m][index:n])
                y1_scli.append(train_y1[m][index:n])
                sentence+=1
                index = n
                
        elif (trainingset[m][n]=="："):
            if (flag == 0) :
                flag = 1
                
            elif n == len(trainingset[m])-1:
                train_scli.append(trainingset[m][index:n])
                y1_scli.append(train_y1[m][index:n])
            
            elif (n - index) >= 100:
                train_scli.append(trainingset[m][index:n])
                y1_scli.append(trainingset[m][index:n])
                sentence+=1
                index = n
                
            else:
                train_scli.append(trainingset[m][index:n-2])
                y1_scli.append(train_y1[m][index:n-2])
                sentence+=1
                index = n-2

In [ ]:
train_x = train_scli[0:int(len(train_scli)*7/10)-1]   
train_y = y1_scli[0:int(len(train_scli)*7/10)-1] 
valid_x = train_scli[int(len(train_scli)*7/10):int(len(train_scli))] 
valid_y = y1_scli[int(len(train_scli)*7/10):int(len(train_scli))]
print(f"train data count: {len(train_x)}")
print(f"validate data count: {len(valid_x)}")
print(f"test data count: {len(test_x)}")


bert_embed = BertEmbedding('chinese_L-12_H-768_A-12',
                           sequence_length=102)

model = BiLSTM_CRF_Model(bert_embed)
model.fit(train_x,
          train_y,
          x_validate=valid_x,
          y_validate=valid_y,
          epochs=40,
          batch_size=256)
model.save('BiLSTM_CRF_Model_2.h5')
model.evaluate(test_x, test_y)

2020-12-28 09:40:01,405 [DEBUG] kashgari - ------------------------------------------------
2020-12-28 09:40:01,408 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-28 09:40:01,409 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-28 09:40:01,413 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-28 09:40:01,415 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-28 09:40:01,417 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

train data count: 18658
validate data count: 7997
test data count: 4443


Preparing text vocab dict: 100%|██████████| 7997/7997 [00:00<00:00, 323738.02it/s]
2020-12-28 09:40:01,612 [DEBUG] kashgari - --- Build vocab dict finished, Total: 28 ---
2020-12-28 09:40:01,612 [DEBUG] kashgari - Top-10: ['[PAD]', 'O', 'I-time', 'B-time', 'I-med_exam', 'I-name', 'I-location', 'B-med_exam', 'I-money', 'B-location']
Calculating sequence length: 100%|██████████| 7997/7997 [00:00<00:00, 1002536.06it/s]
2020-12-28 09:40:09,620 [DEBUG] kashgari - Calculated sequence length = 47
2020-12-28 09:40:11,475 [DEBUG] kashgari - fit input shape: (2, 256, 47)
2020-12-28 09:40:11,476 [DEBUG] kashgari - fit input shape: (256, 47)


Epoch 1/40
72/72 [==============================] - 116s 1s/step - loss: 3.4134 - accuracy: 0.8708 - val_loss: 44.7194 - val_accuracy: 0.9803
Epoch 2/40
72/72 [==============================] - 93s 1s/step - loss: 0.4092 - accuracy: 0.9835 - val_loss: 44.5789 - val_accuracy: 0.9827
Epoch 3/40
72/72 [==============================] - 93s 1s/step - loss: 0.2852 - accuracy: 0.9856 - val_loss: 44.2736 - val_accuracy: 0.9852
Epoch 4/40
72/72 [==============================] - 93s 1s/step - loss: 0.2332 - accuracy: 0.9878 - val_loss: 44.1297 - val_accuracy: 0.9857
Epoch 5/40
72/72 [==============================] - 93s 1s/step - loss: 0.2040 - accuracy: 0.9892 - val_loss: 43.8700 - val_accuracy: 0.9860
Epoch 6/40
72/72 [==============================] - 93s 1s/step - loss: 0.1878 - accuracy: 0.9896 - val_loss: 43.9283 - val_accuracy: 0.9867
Epoch 7/40
72/72 [==============================] - 93s 1s/step - loss: 0.1715 - accuracy: 0.9899 - val_loss: 43.6906 - val_accuracy: 0.9871
Epoch 8/40
7

2020-12-28 10:42:30,385 [INFO] kashgari - model saved to /content/drive/My Drive/AiCup-1/BiLSTM_CRF_Model_2.h5
2020-12-28 10:42:30,387 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 102
2020-12-28 10:42:30,436 [DEBUG] kashgari - predict seq_length: None, input: (2, 4443, 102)


139/139 [==============================] - 46s 300ms/step


2020-12-28 10:43:16,909 [DEBUG] kashgari - predict output: (4443, 102)
2020-12-28 10:43:16,911 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 2 1 0]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]



              precision    recall  f1-score   support

        time     0.6821    0.7601    0.7190       446
    med_exam     0.5931    0.8113    0.6853       106
    location     0.8333    0.7143    0.7692        56
        name     0.5556    0.7576    0.6410        33
       money     0.7049    0.7544    0.7288        57
          ID     0.0000    0.0000    0.0000         9
     contact     0.0000    0.0000    0.0000         2
organization     0.0000    0.0000    0.0000         5
  profession     1.0000    0.3333    0.5000         3

   micro avg     0.6560    0.7448    0.6976       717
   macro avg     0.6628    0.7448    0.6982       717



{'detail': {'ID': {'f1-score': 0, 'precision': 0, 'recall': 0.0, 'support': 9},
  'contact': {'f1-score': 0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
  'location': {'f1-score': 0.7692307692307692,
   'precision': 0.8333333333333334,
   'recall': 0.7142857142857143,
   'support': 56},
  'med_exam': {'f1-score': 0.6852589641434264,
   'precision': 0.593103448275862,
   'recall': 0.8113207547169812,
   'support': 106},
  'money': {'f1-score': 0.7288135593220338,
   'precision': 0.7049180327868853,
   'recall': 0.7543859649122807,
   'support': 57},
  'name': {'f1-score': 0.6410256410256411,
   'precision': 0.5555555555555556,
   'recall': 0.7575757575757576,
   'support': 33},
  'organization': {'f1-score': 0, 'precision': 0, 'recall': 0.0, 'support': 5},
  'profession': {'f1-score': 0.5,
   'precision': 1.0,
   'recall': 0.3333333333333333,
   'support': 3},
  'time': {'f1-score': 0.71898197242842,
   'precision': 0.682092555331992,
   'recall': 0.7600896860986547,
   'support': 4

In [ ]:
model.save('BiLSTM_CRF_Model2.h5')

2020-12-28 07:13:06,736 [INFO] kashgari - model saved to /content/drive/My Drive/AiCup-1/BiLSTM_CRF_Model.h5


'/content/drive/My Drive/AiCup-1/BiLSTM_CRF_Model.h5'